In [1]:
#But sometimes people are willing to accept answers provided within certain timeframe,
#so then the question comes down to choosing between answers provided by different users.
#so say a user is willing to accept answers provided within one week, how to distinguish answers?
#how many answers provided by expert vs. others (so there is maximum likelihood to solve the question)? 

import random
import os
import dill
import operator
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from datetime import datetime
import networkx as nx

os.chdir('/Users/longy/Documents/TDI/capstone')

In [ ]:
names = ['source', 'target', 'time']
df = pd.read_csv('sx-stackoverflow.txt', sep = ' ', header=None, names = names)
a2q = pd.read_csv('sx-stackoverflow-a2q.txt', sep = ' ', header=None, names = names)
c2a = pd.read_csv('sx-stackoverflow-c2a.txt', sep = ' ', header=None, names = names)
c2q = pd.read_csv('sx-stackoverflow-c2q.txt', sep = ' ', header=None, names = names)
a2q['type'] = 1 #answer to questions
c2a['type'] = 2 #comments to answers
c2q['type'] = 3 #comments to questions
#merge a2q, c2a, c2q to main to add type column
df_a2q = df.merge(a2q, on=names, how='left')
df_a2q_c2a = df_a2q.merge(c2a, on=names,how='left')
df_all = df_a2q_c2a.merge(c2q, on=names,how='left')
df_all['type'] = df_all.type_x.fillna(0) + df_all.type_y.fillna(0) + df_all.type.fillna(0)
network = df[['source', 'target', 'time']]
dill.dump(network, open('network.pkd', 'w'))

In [2]:
network = dill.load(open('network.pkd', 'r'))

In [3]:
network.head()

,source,target,time
0,9,8,1217567877
1,1,1,1217573801
2,13,1,1217606247
3,17,1,1217617639
4,48,2,1217618182


In [ ]:
def makeEdge(a,b):
    return a,b
nodes = np.unique(np.concatenate((network.source.unique(), network.target.unique())))
edges = network.apply(lambda row: makeEdge(row['source'], row['target']), axis = 1)

In [ ]:
dill.dump(nodes, open('nodes.pkd', 'w'))
dill.dump(edges, open('edges.pkd', 'w'))

In [ ]:
G=nx.MultiDiGraph()
G.add_nodes_from(nodes)
G.add_edges_from(edges)

In [ ]:
edges.head()